# Data Challenge 1 - Breast Cancer Awareness

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline

## 1. Load the Data

In [3]:
# Load all three dataframes with the interaction data.
df_user = pd.read_csv('yammer_users.csv')
print(df_user.shape)
df_user.head()

(19066, 6)


,user_id,created_at,company_id,language,activated_at,state
0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
1,1.0,2013-01-01 13:07:46,28.0,english,NaN,pending
2,2.0,2013-01-01 10:59:05,51.0,english,NaN,pending
3,3.0,2013-01-01 18:40:36,2800.0,german,2013-01-01 18:42:02,active
4,4.0,2013-01-01 14:37:51,5110.0,indian,2013-01-01 14:39:05,active


In [36]:
# Filter to only show engagement.
df_event = pd.read_csv('yammer_events.csv')
df_engage_filter = df_event['event_type'] == 'engagement'
df_engage = df_event[df_engage_filter]
print(df_engage.shape)
df_engage.head()

(321575, 7)


,user_id,occurred_at,event_type,event_name,location,device,user_type
0,10522.0,2014-05-02 11:02:39,engagement,login,Japan,dell inspiron notebook,3.0
1,10522.0,2014-05-02 11:02:53,engagement,home_page,Japan,dell inspiron notebook,3.0
2,10522.0,2014-05-02 11:03:28,engagement,like_message,Japan,dell inspiron notebook,3.0
3,10522.0,2014-05-02 11:04:09,engagement,view_inbox,Japan,dell inspiron notebook,3.0
4,10522.0,2014-05-02 11:03:16,engagement,search_run,Japan,dell inspiron notebook,3.0


In [5]:
df_emails = pd.read_csv('yammer_emails.csv')
print(df_emails.shape)
df_emails.head()

(90389, 4)


,user_id,occurred_at,action,user_type
0,0.0,2014-05-06 09:30:00,sent_weekly_digest,1.0
1,0.0,2014-05-13 09:30:00,sent_weekly_digest,1.0
2,0.0,2014-05-20 09:30:00,sent_weekly_digest,1.0
3,0.0,2014-05-27 09:30:00,sent_weekly_digest,1.0
4,0.0,2014-06-03 09:30:00,sent_weekly_digest,1.0


In [37]:
df_emails['action'].unique()

array(['sent_weekly_digest', 'email_open', 'email_clickthrough',
       'sent_reengagement_email'], dtype=object)

In [12]:
df_time = pd.read_csv('dimension_rollup_periods.csv')
print(df_time.shape)
df_time.info()

(56002, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56002 entries, 0 to 56001
Data columns (total 6 columns):
period_id    56002 non-null float64
time_id      56002 non-null object
pst_start    56002 non-null object
pst_end      56002 non-null object
utc_start    56002 non-null object
utc_end      56002 non-null object
dtypes: float64(1), object(5)
memory usage: 2.6+ MB


In [13]:
df_time['period_id'].unique()

array([1.000e+00, 2.000e+00, 1.007e+03, 1.028e+03, 2.001e+03, 2.007e+03])

In [27]:
df_period_filter = df_time['period_id'] == 1007
df_time_7 = df_time[df_period_filter]
df_time_7.head()

,period_id,time_id,pst_start,pst_end,utc_start,utc_end
1252,1007.0,2013-01-01 00:00:00,2012-12-25 00:00:00,2013-01-01 00:00:00,2012-12-25 08:00:00,2013-01-01 08:00:00
1253,1007.0,2013-01-02 00:00:00,2012-12-26 00:00:00,2013-01-02 00:00:00,2012-12-26 08:00:00,2013-01-02 08:00:00
1254,1007.0,2013-01-03 00:00:00,2012-12-27 00:00:00,2013-01-03 00:00:00,2012-12-27 08:00:00,2013-01-03 08:00:00
1255,1007.0,2013-01-04 00:00:00,2012-12-28 00:00:00,2013-01-04 00:00:00,2012-12-28 08:00:00,2013-01-04 08:00:00
1256,1007.0,2013-01-05 00:00:00,2012-12-29 00:00:00,2013-01-05 00:00:00,2012-12-29 08:00:00,2013-01-05 08:00:00


In [33]:
# Filter it to only include the dates in the summer 2014.
df_time_summer14 = df_time.loc[1737:1859]
print(df_time_summer14.shape)
df_time_summer14.head()

(123, 6)


,period_id,time_id,pst_start,pst_end,utc_start,utc_end
1737,1007.0,2014-05-01 00:00:00,2014-04-24 00:00:00,2014-05-01 00:00:00,2014-04-24 07:00:00,2014-05-01 07:00:00
1738,1007.0,2014-05-02 00:00:00,2014-04-25 00:00:00,2014-05-02 00:00:00,2014-04-25 07:00:00,2014-05-02 07:00:00
1739,1007.0,2014-05-03 00:00:00,2014-04-26 00:00:00,2014-05-03 00:00:00,2014-04-26 07:00:00,2014-05-03 07:00:00
1740,1007.0,2014-05-04 00:00:00,2014-04-27 00:00:00,2014-05-04 00:00:00,2014-04-27 07:00:00,2014-05-04 07:00:00
1741,1007.0,2014-05-05 00:00:00,2014-04-28 00:00:00,2014-05-05 00:00:00,2014-04-28 07:00:00,2014-05-05 07:00:00


## 2. Join the User Information to the Events and Emails Dataframes

In [38]:
# Use Pandas join to join df_user and df_event dataframes.

df_engage_user = df_engage.join(df_user,on='user_id',lsuffix='_engage', rsuffix='_users')
df_engage_user.head()

,user_id_engage,occurred_at,event_type,event_name,location,device,user_type,user_id_users,created_at,company_id,language,activated_at,state
0,10522.0,2014-05-02 11:02:39,engagement,login,Japan,dell inspiron notebook,3.0,10522.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
1,10522.0,2014-05-02 11:02:53,engagement,home_page,Japan,dell inspiron notebook,3.0,10522.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
2,10522.0,2014-05-02 11:03:28,engagement,like_message,Japan,dell inspiron notebook,3.0,10522.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
3,10522.0,2014-05-02 11:04:09,engagement,view_inbox,Japan,dell inspiron notebook,3.0,10522.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active
4,10522.0,2014-05-02 11:03:16,engagement,search_run,Japan,dell inspiron notebook,3.0,10522.0,2014-04-04 16:48:03,1147.0,japanese,2014-04-04 16:49:36,active


In [8]:
# Use Pandas join to join df_user and df_emails dataframes.

df_email_user = df_emails.join(df_user,on='user_id',lsuffix='_events', rsuffix='_users')
df_email_user.head()

,user_id_events,occurred_at,action,user_type,user_id_users,created_at,company_id,language,activated_at,state
0,0.0,2014-05-06 09:30:00,sent_weekly_digest,1.0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
1,0.0,2014-05-13 09:30:00,sent_weekly_digest,1.0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
2,0.0,2014-05-20 09:30:00,sent_weekly_digest,1.0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
3,0.0,2014-05-27 09:30:00,sent_weekly_digest,1.0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
4,0.0,2014-06-03 09:30:00,sent_weekly_digest,1.0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
